In [3]:
"""
This program is to conver the dataset into required format more or less to clean the data and prepare it for training and testing
"""
import os

#  To save or load dataset or load model or dump model
import pickle

# Medaipipe library is used to segment the captured image and get the hand and finger mapped images
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt

# Storing all mp methods/object into variables 
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Creatting instance of hand capturing method
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

# Location of dataset
DATA_DIR = './data'

data = []
labels = []
# Looping to acess the dataset
for dir_ in os.listdir(DATA_DIR):
    for img_path in os.listdir(os.path.join(DATA_DIR, dir_)):
        data_aux = []

        x_ = []
        y_ = []

        img = cv2.imread(os.path.join(DATA_DIR, dir_, img_path))
        # Converting image into RGB vectors
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Determining the location of hands
        results = hands.process(img_rgb)

        # Checking for multiple hand location
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y

                    # Storing the hand locations in lists
                    x_.append(x)
                    y_.append(y)

                # Calculating the distance from the lowest point
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    data_aux.append(x - min(x_))
                    data_aux.append(y - min(y_))

            # Storing hand locations and labeling it according to the type of dataset 
            data.append(data_aux)
            labels.append(dir_)

# Storing this data and label pair in another file
f = open('data.pickle', 'wb')

# Data would be 42 length list and label would be a single value of directory
pickle.dump({'data': data, 'labels': labels}, f)
f.close()